Exploratory analysis of the review data to find:
* When is the time in which most reviews are written?

In [1]:
import pyodbc
dwh_login = {'option': '3',
             'database': 'datawarehouse',
             'driver': '{PostgreSQL ANSI}',
             'server': 'je-dw.cspauv1pfeh3.eu-west-1.redshift.amazonaws.com',
             'pwd': '',
             'user': '',
             'port': '5439'}

sql = 'select 1;'

try:
    schema_conn = pyodbc.connect('DSN=Redshift x64')
    schema_db = schema_conn.cursor()
    data = schema_db.execute(sql).fetchall()
    schema_conn.commit()
#    schema_db.close()
#    schema_conn.close()
except Exception, ex:
    raise

print data

[(1, )]


In [6]:
#What is the number of customer reviews per year in the ecom.rating table
%matplotlib inline
from scipy.stats import kde
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
q="""
select datepart(hour,rate_date) as hour, datepart(hour,rate_date)*60+datepart(min,rate_date) as total_min from ecom.rating where commentary<>'' and country_key=8
ORDER BY RANDOM()
limit 1000
"""
df = pd.read_sql(q, schema_conn)

### plotting (hours) ###
#kernel density stimation (non-parametric prob distrib)
NN=30
x1=np.asarray(df["hour"])
density1 = kde.gaussian_kde(x1)
xgrid1 = np.linspace(x1.min(), x1.max(), NN)

plt.title("Probability distribution of customer reviews",fontsize=24)
plt.xlabel("Time (in hours)",fontsize=20)
plt.ylabel("Prob. Density Function",fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.plot(xgrid1, density1(xgrid1), 'r-',linewidth=2.0)
plt.plot(x1, np.zeros(x1.shape), 'r+', ms=12) #rug plot

plt.show()

### plotting (min) ###
#kernel density stimation (non-parametric prob distrib)
NN=100
x1=np.asarray(df["total_min"])
density1 = kde.gaussian_kde(x1)
xgrid1 = np.linspace(x1.min(), x1.max(), NN)

plt.title("Probability distribution of customer reviews")
plt.xlabel("Time (in total minutes)")
plt.ylabel("Prob. Density Function")
plt.plot(xgrid1, density1(xgrid1), 'b-')
plt.plot(x1, np.zeros(x1.shape), 'b+', ms=12) #rug plot
plt.show()
